In [1]:
import torch
import torch.nn as nn
import torchvision

import io

import pandas as pd

In [2]:
import sklearn.metrics as metrics

In [3]:
device = 'cuda'

Carrega uma rede ResNet18 pré treinada no ImageNet

In [4]:
resnet18 = torchvision.models.resnet18(pretrained=True)
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
# Congela o treinamento para todas as camadas de "features"
for param in resnet18.parameters():
    param.requires_grad = False
    
num_features = resnet18.fc.in_features

resnet18.fc = nn.Linear(num_features, 2)

In [4]:
transform = torchvision.transforms.ToTensor()
DeepFakeDataset = torchvision.datasets.ImageFolder('./Faces Dataset/224px/', transform=transform)

In [5]:
from torch.utils.data.sampler import SubsetRandomSampler

percentage_for_train = 0.7

shuffle_indices = torch.randperm(len(DeepFakeDataset))
train_indices = shuffle_indices[:int(percentage_for_train*len(DeepFakeDataset))]
val_indices = shuffle_indices[int(percentage_for_train*len(DeepFakeDataset)):]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_dataloader = torch.utils.data.DataLoader(DeepFakeDataset, batch_size=32, sampler=train_sampler)
test_dataloader = torch.utils.data.DataLoader(DeepFakeDataset, batch_size=32, sampler=val_sampler)

In [8]:
import torch.nn.functional as F

def training_loop(n_epochs, model, loss_function, optimizer):
    model = model.to(device)
    loss_function = loss_function.to(device)
    
    model.train()
    for epoch in range(0, n_epochs):
        loss_sum = 0
        iteration = 0
        print("Beggining epoch {}...".format(epoch+1))
        for images, labels in train_dataloader:
            
            images = images.to(device)
            labels = labels.to(device)
            
            predictions = model(images)
            loss = loss_function(predictions, labels.long())
            loss_sum += loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            iteration += 1
            
            if iteration % 20 == 0:
                print("Iteration {} Loss {:.5f}".format(iteration, loss_sum.item()/20))
                loss_sum = 0

## Inicialmente, rodaremos o modelo cru no dataset para checar seu desempenho

In [9]:
def eval_loop(model):
    all_labels = torch.LongTensor([]).cuda()
    all_predictions = torch.LongTensor([]).cuda()
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            predictions = model(images)
            predictions = predictions.max(dim=1)[1]
            all_predictions = torch.cat((all_predictions, predictions))
            all_labels = torch.cat((all_labels, labels))
            
    return all_predictions, all_labels

In [10]:
def createDataFrame_conf_matrix(all_pread, all_labels):
    c_mat = pd.DataFrame(metrics.confusion_matrix(all_labels.cpu().detach().numpy(), all_pred.cpu().detach().numpy()))
    c_mat = c_mat.rename(columns={0: 'FAKE_predicted', 1: "REAL_predicted"}, index={0: 'FAKE', 1: "REAL"})
    rows = [c_mat.loc['FAKE'][0] / c_mat.sum()[0] * 100, c_mat.loc['REAL'][1] / c_mat.sum()[1] * 100]
    columns = [c_mat['FAKE_predicted'][0] / c_mat.sum(axis=1)[0] * 100, c_mat['REAL_predicted'][1] / c_mat.sum(axis=1)[1] * 100]
    accuracy = (c_mat.loc['FAKE'][0] + c_mat.loc['REAL'][1]) / (c_mat.sum(axis=1)[0] + c_mat.sum(axis=1)[1])*100 
    c_mat.loc['Percentage'] = [str(round(rows[0],2)) +' %', str(round(rows[1],2)) +' %']
    c_mat['Percentage'] = [str(round(columns[0],2)) +' %', str(round(columns[1],2)) +' %', str(round(accuracy,2)) +' %']
    return c_mat

In [11]:
all_pred, all_labels = eval_loop(resnet18)

In [12]:
all_pred = all_pred.to(dtype=torch.int8)
all_labels = all_labels.to(dtype=torch.int8)

In [13]:
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,12103,162,98.68 %
REAL,6499,210,3.13 %
Percentage,65.06 %,56.45 %,64.89 %


# Agora, treinamos o modelo e testamos novamente

In [14]:
loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(resnet18.fc.parameters(), lr=1e-4)

training_loop(n_epochs=2, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
Iteration 20 Loss 0.67643
Iteration 40 Loss 0.65845
Iteration 60 Loss 0.66034
Iteration 80 Loss 0.64786
Iteration 100 Loss 0.63336
Iteration 120 Loss 0.63432
Iteration 140 Loss 0.62661
Iteration 160 Loss 0.62194
Iteration 180 Loss 0.59436
Iteration 200 Loss 0.62452
Iteration 220 Loss 0.60225
Iteration 240 Loss 0.61013
Iteration 260 Loss 0.61110
Iteration 280 Loss 0.59350
Iteration 300 Loss 0.59040
Iteration 320 Loss 0.58194
Iteration 340 Loss 0.58583
Beggining epoch 2...
Iteration 20 Loss 0.58576
Iteration 40 Loss 0.58914
Iteration 60 Loss 0.58015
Iteration 80 Loss 0.56742
Iteration 100 Loss 0.57734
Iteration 120 Loss 0.56756
Iteration 140 Loss 0.57612
Iteration 160 Loss 0.54798
Iteration 180 Loss 0.56418
Iteration 200 Loss 0.56996
Iteration 220 Loss 0.57416
Iteration 240 Loss 0.57541
Iteration 260 Loss 0.56175
Iteration 280 Loss 0.56356
Iteration 300 Loss 0.56534
Iteration 320 Loss 0.56293
Iteration 340 Loss 0.55630


In [16]:
loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(resnet18.fc.parameters(), lr=1e-4)

training_loop(n_epochs=2, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
Iteration 20 Loss 0.56275
Iteration 40 Loss 0.56207
Iteration 60 Loss 0.55060
Iteration 80 Loss 0.54214
Iteration 100 Loss 0.54087
Iteration 120 Loss 0.53913
Iteration 140 Loss 0.55167
Iteration 160 Loss 0.55006
Iteration 180 Loss 0.55654
Iteration 200 Loss 0.54005
Iteration 220 Loss 0.55786
Iteration 240 Loss 0.53936
Iteration 260 Loss 0.53492
Iteration 280 Loss 0.54099
Iteration 300 Loss 0.55904
Iteration 320 Loss 0.53747
Iteration 340 Loss 0.52063
Beggining epoch 2...
Iteration 20 Loss 0.52356
Iteration 40 Loss 0.52971
Iteration 60 Loss 0.53135
Iteration 80 Loss 0.54012
Iteration 100 Loss 0.54560
Iteration 120 Loss 0.53001
Iteration 140 Loss 0.52491
Iteration 160 Loss 0.53397
Iteration 180 Loss 0.54046
Iteration 200 Loss 0.52991
Iteration 220 Loss 0.52917
Iteration 240 Loss 0.53029
Iteration 260 Loss 0.53438
Iteration 280 Loss 0.53549
Iteration 300 Loss 0.52045
Iteration 320 Loss 0.54157
Iteration 340 Loss 0.54018


In [17]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,10911,1354,88.96 %
REAL,3632,3077,45.86 %
Percentage,75.03 %,69.44 %,73.72 %


In [18]:
resnet18 = torchvision.models.resnet18(pretrained=True)

# Congela o treinamento para todas as camadas de "features"
for param in resnet18.parameters():
    param.requires_grad = False
    
num_features = resnet18.fc.in_features

resnet18.fc = nn.Linear(num_features, 2)

# Aqui estou experimentado colocar um peso 3x maior para a classe de não deepfake
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 2.0]))

optimizer = torch.optim.Adam(resnet18.fc.parameters(), lr=1e-4)

training_loop(n_epochs=3, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
Iteration 20 Loss 0.72256
Iteration 40 Loss 0.71473
Iteration 60 Loss 0.70292
Iteration 80 Loss 0.68108
Iteration 100 Loss 0.67987
Iteration 120 Loss 0.67782
Iteration 140 Loss 0.67009
Iteration 160 Loss 0.66080
Iteration 180 Loss 0.65615
Iteration 200 Loss 0.65454
Iteration 220 Loss 0.64711
Iteration 240 Loss 0.63942
Iteration 260 Loss 0.63260
Iteration 280 Loss 0.63503
Iteration 300 Loss 0.62150
Iteration 320 Loss 0.63241
Iteration 340 Loss 0.62298
Beggining epoch 2...
Iteration 20 Loss 0.62177
Iteration 40 Loss 0.61201
Iteration 60 Loss 0.60821
Iteration 80 Loss 0.61511
Iteration 100 Loss 0.60929
Iteration 120 Loss 0.60121
Iteration 140 Loss 0.60564
Iteration 160 Loss 0.61737
Iteration 180 Loss 0.60660
Iteration 200 Loss 0.59891
Iteration 220 Loss 0.60599
Iteration 240 Loss 0.59215
Iteration 260 Loss 0.60073
Iteration 280 Loss 0.58796
Iteration 300 Loss 0.58770
Iteration 320 Loss 0.60131
Iteration 340 Loss 0.57767
Beggining epoch 3...
Iteration 20 Loss 0.58587
I

In [19]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,7213,5052,58.81 %
REAL,1267,5442,81.11 %
Percentage,85.06 %,51.86 %,66.7 %


In [20]:
training_loop(n_epochs=3, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
Iteration 20 Loss 0.57211
Iteration 40 Loss 0.56457
Iteration 60 Loss 0.55785
Iteration 80 Loss 0.56494
Iteration 100 Loss 0.56019
Iteration 120 Loss 0.56736
Iteration 140 Loss 0.56689
Iteration 160 Loss 0.57515
Iteration 180 Loss 0.56002
Iteration 200 Loss 0.56545
Iteration 220 Loss 0.55541
Iteration 240 Loss 0.54879
Iteration 260 Loss 0.57667
Iteration 280 Loss 0.55515
Iteration 300 Loss 0.55830
Iteration 320 Loss 0.57207
Iteration 340 Loss 0.55743
Beggining epoch 2...
Iteration 20 Loss 0.55767
Iteration 40 Loss 0.56051
Iteration 60 Loss 0.56229
Iteration 80 Loss 0.55561
Iteration 100 Loss 0.55647
Iteration 120 Loss 0.54622
Iteration 140 Loss 0.55436
Iteration 160 Loss 0.54590
Iteration 180 Loss 0.55158
Iteration 200 Loss 0.56509
Iteration 220 Loss 0.54617
Iteration 240 Loss 0.54703
Iteration 260 Loss 0.55660
Iteration 280 Loss 0.54843
Iteration 300 Loss 0.55484
Iteration 320 Loss 0.54946
Iteration 340 Loss 0.53637
Beggining epoch 3...
Iteration 20 Loss 0.54547
I

In [21]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,8341,3924,68.01 %
REAL,1583,5126,76.4 %
Percentage,84.05 %,56.64 %,70.98 %


Bem melhor! Infelizmente nossa acurácia geral caiu, mas o modelo conseguiu predizer mais rostos reais como reais e mais rostos falsos como falsos. Ainda há um longo caminho a se percorrer. Por hora, vamos tentar diminuir um pouco o learning rate e aumentar o número de épocas utilizando um modelo mais poderoso e observar os resultados.

In [23]:
resnet50 = torchvision.models.resnet50(pretrained=True)

# Congela o treinamento para todas as camadas de "features"
for param in resnet50.parameters():
    param.requires_grad = False
    
num_features = resnet50.fc.in_features

resnet50.fc = nn.Linear(num_features, 2)

# Aqui estou experimentado colocar um peso 9x maior para a classe de não deepfake
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.5]))

optimizer = torch.optim.Adam(resnet50.fc.parameters(), lr=0.5e-3)

training_loop(n_epochs=3, model=resnet50, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
Iteration 20 Loss 0.68799
Iteration 40 Loss 0.64818
Iteration 60 Loss 0.61646
Iteration 80 Loss 0.60473
Iteration 100 Loss 0.61077
Iteration 120 Loss 0.59524
Iteration 140 Loss 0.58154
Iteration 160 Loss 0.58411
Iteration 180 Loss 0.57724
Iteration 200 Loss 0.57432
Iteration 220 Loss 0.56837
Iteration 240 Loss 0.57737
Iteration 260 Loss 0.55935
Iteration 280 Loss 0.57288
Iteration 300 Loss 0.55380
Iteration 320 Loss 0.57015
Iteration 340 Loss 0.55017
Beggining epoch 2...
Iteration 20 Loss 0.54942
Iteration 40 Loss 0.54488
Iteration 60 Loss 0.54837
Iteration 80 Loss 0.54233
Iteration 100 Loss 0.54652
Iteration 120 Loss 0.54310
Iteration 140 Loss 0.54536
Iteration 160 Loss 0.54697
Iteration 180 Loss 0.53969
Iteration 200 Loss 0.54218
Iteration 220 Loss 0.55252
Iteration 240 Loss 0.54760
Iteration 260 Loss 0.54816
Iteration 280 Loss 0.53299
Iteration 300 Loss 0.53091
Iteration 320 Loss 0.52282
Iteration 340 Loss 0.52542
Beggining epoch 3...
Iteration 20 Loss 0.51310
I

In [26]:
all_pred, all_labels = eval_loop(resnet50)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,9331,2934,76.08 %
REAL,1802,4907,73.14 %
Percentage,83.81 %,62.58 %,75.04 %


Um resultado promissor. Embora nossa acurácia geral tenha aumentado apenas um pouco para 75.04%, fomos capazes de classificar corretamente 76.08% dos deepfakes e 73.14% dos reais. Estamos indo certo em alguma direção!

In [27]:
torch.save(resnet50.state_dict(), './Saved Models/resnet50_pretrained_trainedover4folders_balanced.pt')

#torch.save({
#            'epoch': epoch,
#            'model_state_dict': model.state_dict(),
#            'optimizer_state_dict': optimizer.state_dict(),
#            'loss': loss,
#            ...
#            }, PATH)

In [11]:
resnet50 = torchvision.models.resnet50(pretrained=False)
    
num_features = resnet50.fc.in_features

resnet50.fc = nn.Linear(num_features, 2)

resnet50.load_state_dict(torch.load('./Saved Models/resnet50_pretrained_trainedover4folders_balanced.pt'))

<All keys matched successfully>

In [7]:
# Aqui estou experimentado colocar um peso 9x maior para a classe de não deepfake
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.5]))

optimizer = torch.optim.Adam(resnet50.fc.parameters(), lr=1e-4)

training_loop(n_epochs=5, model=resnet50, loss_function=loss_function, optimizer=optimizer)

NameError: name 'training_loop' is not defined

In [15]:
all_pred, all_labels = eval_loop(resnet50)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,8954,3260,73.31 %
REAL,1426,5334,78.91 %
Percentage,86.26 %,62.07 %,75.3 %


In [12]:
# Congela o treinamento para todas as camadas de "features"

for param in resnet50.parameters():
    param.requires_grad = True

for param in resnet50.fc.parameters():
    param.requires_grad = False
    
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.5]))    
optimizer = torch.optim.Adam(resnet50.parameters(), lr=1e-4)

training_loop(n_epochs=3, model=resnet50, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
Iteration 20 Loss 0.59426
Iteration 40 Loss 0.52556
Iteration 60 Loss 0.41659
Iteration 80 Loss 0.46110
Iteration 100 Loss 0.36822
Iteration 120 Loss 0.37085
Iteration 140 Loss 0.35899
Iteration 160 Loss 0.34524
Iteration 180 Loss 0.29661
Iteration 200 Loss 0.25882
Iteration 220 Loss 0.23727
Iteration 240 Loss 0.29944
Iteration 260 Loss 0.27978
Iteration 280 Loss 0.28345
Iteration 300 Loss 0.23573
Iteration 320 Loss 0.30361
Iteration 340 Loss 0.24410
Iteration 360 Loss 0.23524
Iteration 380 Loss 0.23996
Iteration 400 Loss 0.26933
Iteration 420 Loss 0.21516
Iteration 440 Loss 0.24700
Iteration 460 Loss 0.15264
Iteration 480 Loss 0.23808
Iteration 500 Loss 0.22310
Iteration 520 Loss 0.19420
Iteration 540 Loss 0.21317
Iteration 560 Loss 0.21707
Iteration 580 Loss 0.17902
Iteration 600 Loss 0.19321
Iteration 620 Loss 0.22920
Iteration 640 Loss 0.19598
Iteration 660 Loss 0.19203
Iteration 680 Loss 0.20343
Iteration 700 Loss 0.16264
Iteration 720 Loss 0.15339
Iteration 7

KeyboardInterrupt: 

In [13]:
all_pred, all_labels = eval_loop(resnet50)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,11708,468,96.16 %
REAL,391,6407,94.25 %
Percentage,96.77 %,93.19 %,95.47 %


Eita :|

In [14]:
torch.save(resnet50.state_dict(), './Saved Models/resnet50_pretrained_balanced_unfreezed.pt')